In [1]:
import glob
import random

In [2]:
list_of_txts=glob.glob("C:/anaconda/ep/ep*.txt")

In [3]:
random.seed(1)
s=random.sample(range(0,len(list_of_txts)), round(0.8*(len(list_of_txts))))

In [4]:
 t=list(set(range(0,len(list_of_txts))) - set(s))

In [5]:
test_txts= [list_of_txts[i] for i in t]

In [6]:
content=[]
for file in test_txts:
    fp=open(file, encoding="utf8")
    try:
        for line in fp:
             content.append(line)
    finally:
        fp.close()

In [7]:
import re

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [8]:
no_html_content=[]
for s in content:
     no_html_content.append(cleanhtml(s))

In [9]:
content=[]
for s in no_html_content:
    if s!='\n':
        content.append(s)

In [10]:
content=[s.strip('\n') for s in content]

In [11]:
del no_html_content

In [12]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [13]:
tunigrams=[]
for s in content:
    tunigrams+=tokenizer.tokenize(s)

In [14]:
tunigrams=[x for x in map(str.lower,tunigrams)]

In [18]:
from nltk.util import ngrams
tbigrams = [ gram for gram in ngrams(tunigrams, 2) ]

In [84]:
import numpy as np
count = np.load('unigrams.npy').item()

In [85]:
count1 = np.load('bigrams.npy').item()

In [86]:
count2= np.load('trigrams.npy').item()

In [30]:
def trimodel(tup):
    trikeys,bikeys,unikeys=count2.keys(),count1.keys(),count.keys()
    if tup[0:2] in bikeys:
        if tup in trikeys:
             p=(count2[tup]+1)/(count1[tup[0:2]]+len(count))
        else :
             p=1/(count1[tup[0:2]]+len(count))
    else : 
        if tup[1:] in bikeys:
             p=0.4*count1[tup[1:]]/count[tup[1]]
        else:
            if tup[2] in unikeys:
                 p=0.4*count[tup[2]]/sum(count.values())
            else :
                 p=10**(-15)
    return p     

In [31]:
def bimodel(tup):
    bikeys,unikeys=count1.keys(),count.keys()
    if tup[0] in unikeys:
        if tup in bikeys:
             p=(count1[tup]+1)/(count[tup[0]]+len(count))
        else :
             p=1/(count[tup[0]]+len(count))
    else : 
        if tup[1] in unikeys:
            p=0.4*(count[tup[1]]/sum(count.values()))
        else: 
            p=10**(-15)
    return p 

In [88]:
from collections import Counter
import operator
def predict_next_word(test_word):
    prev_word=[key for key in count1.keys() if key[0]==test_word ]
    prob_per_next_word={word[1]:bimodel(word) for word in prev_word}
    if len(prob_per_next_word)>0:
            most_prob_words=dict(Counter(prob_per_next_word).most_common(3))
            most_prob_words = sorted(most_prob_words.items(), key=operator.itemgetter(1),reverse=True)
            return most_prob_words         
    else: 
            next_word=''
            prob='unknown'
            return next_word+' '+str(prob)

In [89]:
def predict_next_word1(test_bi):
    prev_word=[key for key in count2.keys() if key[0:2]==test_bi ]
    prob_per_next_word={tri[2]:trimodel(tri) for tri in prev_word}
    if len(prob_per_next_word)>0:
            most_prob_words=dict(Counter(prob_per_next_word).most_common(3))
            most_prob_words = sorted(most_prob_words.items(), key=operator.itemgetter(1),reverse=True)
            return most_prob_words 
    else:
        next_word=''
        prob='unknown'
        return next_word+' '+str(prob)

In [90]:
predict_next_word('european')

[('union', 0.283920344522576),
 ('parliament', 0.1315216982968536),
 ('commission', 0.03497136378073228)]

In [91]:
predict_next_word('approval')

[('of', 0.05232179920076825),
 ('and', 0.0041820265791022585),
 ('to', 0.004089092655122208)]

In [92]:
predict_next_word('intelligence')

[('services', 0.004733887874484344),
 ('and', 0.00375329681476973),
 ('of', 0.0010144045445323595)]

In [77]:
predict_next_word('pension')

[('funds', 0.010978334790634411),
 ('schemes', 0.007545581139285599),
 ('systems', 0.006314971339745458)]

In [93]:
predict_next_word1(('pension','funds'))

[('and', 0.0017497512608501732),
 ('in', 0.0009263389028030329),
 ('are', 0.0008577212062991045)]

In [94]:
predict_next_word1(('extremely','important'))

[('for', 0.010800627943485087),
 ('that', 0.01010989010989011),
 ('to', 0.009419152276295133)]

In [103]:
predict_next_word1(('rest', 'of'))

[('the', 0.05144238517324738),
 ('europe', 0.006865431103948429),
 ('us', 0.0027397260273972603)]

In [107]:
predict_next_word1(('council', 'is'))

[('not', 0.011117374420643868),
 ('aware', 0.0029750720280596267),
 ('also', 0.0028498058374044847)]